In [1]:
import pandas as pd
import numpy as np
import gspread
from pydata_google_auth import get_user_credentials
from google.cloud import bigquery
from pycaret.clustering import *

project_id = 'ledger-fcc1e'
creds = get_user_credentials([
    'https://www.googleapis.com/auth/cloud-platform', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive',
])
client = bigquery.Client(project=project_id, credentials=creds)

gc = gspread.authorize(creds) 
pull_data = lambda query: client.query(query).result().to_dataframe()
get_worksheet = lambda key, name: gc.open_by_key(key).worksheet(name)

In [2]:
## -- QUERY TO GET IMPRESSIONS COUNT
imp_query =\
'''
WITH users AS (
  SELECT DISTINCT user_id
  FROM `ledger-fcc1e.dg_buku_trnsfmd.ppm_table`
  WHERE last_transaction_date >= '2023-04-01'
)

, loyalty AS (
  SELECT user_id, date_
  FROM `ledger-fcc1e.trb_mxg_reports.loyalty_ssot`
  WHERE 
    date_ IN ('2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01')
    AND kyc_tier <> 'NON_KYC'
)

, impressions AS (
  SELECT
    SUBSTR(CAST(profile_phone AS STRING), 3) AS user_id,
    DATE_TRUNC(notification_date, MONTH) AS date_,
    SUM(pn_count) AS send_count,
    SUM(impressions_count) AS impressions_count,
    SUM(clicks_count) AS read_count,
    COUNT(DISTINCT user_activated) AS activated_count,
    COUNT(DISTINCT user_converted) AS converted_count
  FROM `ledger-fcc1e.crm_reports.crm_campaign_performance`
  WHERE 
    notification_date BETWEEN '2023-05-01' AND '2023-08-31'
    AND objective_code IN ('PAY_GEN', 'PAY_POUT', 'PPOB_PRE', 'PPOB_GEN', 'PAY', 'PPOB', 'SAL_TOPUP')
  GROUP BY 1, 2
)

SELECT
  user_id,
  date_,
  CASE WHEN send_count > 0 THEN 1 ELSE IFNULL(send_count, 0) END AS send_count,
  CASE WHEN impressions_count > 0 THEN 1 ELSE IFNULL(impressions_count, 0) END AS impressions_count,
  CASE WHEN read_count > 0 THEN 1 ELSE IFNULL(read_count, 0) END AS read_count,
  IFNULL(activated_count, 0) AS activated_count,
  IFNULL(converted_count, 0) AS converted_count
FROM users
INNER JOIN loyalty USING (user_id)
LEFT JOIN impressions USING (user_id, date_)
WHERE send_count IS NOT NULL
'''
imp = client.query(imp_query).result().to_dataframe()
imp.to_pickle('data/deploy_imp_v2.pkl')


imp = pd.read_pickle('data/deploy_imp_v2.pkl')

In [3]:
# Load model and training data
kmeans = load_model('model3')
raw = pd.read_pickle('data/raw_data_v2.pkl')
df =\
(
    raw
    .assign(
        # referee_count = lambda x: x.referee_count.fillna(0),
        date_ = lambda x: x.date_.astype(str),
        index_ = lambda x: x.user_id + " " + x.date_,

        # M1_core_count = lambda x: x.M1_core_count/std_core,
        # M1_acc_count = lambda x: x.M1_acc_count/std_acc,
        # M1_ppob_count = lambda x: x.M1_ppob_count/std_ppob,
        # M1_campaign_saldo = lambda x: x.M1_campaign_saldo/std_saldo,
        # age_on_platform_days = lambda x: x.age_on_platform_days/std_age,
        M1_core_count = lambda x: np.log1p(x.M1_core_count),
        M1_ppob_count = lambda x: np.log1p(x.M1_ppob_count),
        M1_acc_count = lambda x: np.log1p(x.M1_acc_count),
        M1_campaign_saldo = lambda x: np.log1p(x.M1_campaign_saldo),
        age_on_platform_days = lambda x: np.log1p(x.age_on_platform_days),

        is_send_m1 = lambda x: x.is_send_m1.fillna(0),
        is_imp_m1 = lambda x: x.is_imp_m1.fillna(0),
        is_read_m1 = lambda x: x.is_read_m1.fillna(0),
        is_act_m1 = lambda x: x.is_act_m1.fillna(0),
        is_conv_m1 = lambda x: x.is_conv_m1.fillna(0),
    )
    .drop(columns=['user_id', 'date_', 'is_send_m1', 'is_imp_m1', 'is_read_m1', 'is_act_m1', 'is_conv_m1', 'M1_campaign_saldo'])
    # .drop(columns=['user_id', 'date_', 'M1_core_count', 'M1_acc_count', 'M1_ppob_count'])
    # .drop(columns=['user_id', 'date_'])
    .set_index('index_')
)

Transformation Pipeline and Model Successfully Loaded


In [4]:
# setup model
s = setup(
    df, 
    normalize=True, 
    ignore_features=['user_id'],
    use_gpu = True,
    session_id = 123)

,Description,Value
0,Session id,123
1,Original data shape,"(203288, 10)"
2,Transformed data shape,"(203288, 27)"
3,Ignore features,1
4,Ordinal features,1
5,Numeric features,6
6,Categorical features,4
7,Rows with missing values,56.8%
8,Preprocess,True
9,Imputation type,simple


In [5]:
imp_df =\
(
    imp
    .assign(
        date_ = lambda x: x.date_.astype(str)
    )
)

In [6]:
imp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127650 entries, 0 to 127649
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   user_id            127650 non-null  object
 1   date_              127650 non-null  object
 2   send_count         127650 non-null  Int64 
 3   impressions_count  127650 non-null  Int64 
 4   read_count         127650 non-null  Int64 
 5   activated_count    127650 non-null  Int64 
 6   converted_count    127650 non-null  Int64 
dtypes: Int64(5), object(2)
memory usage: 7.4+ MB


In [7]:
imp_df.describe()

,send_count,impressions_count,read_count,activated_count,converted_count
count,127650.0,127650.0,127650.0,127650.0,127650.0
mean,1.0,0.918841,0.106761,0.677282,0.317705
std,0.0,0.273081,0.30881,0.467736,0.465687
min,1.0,0.0,0.0,0.0,0.0
25%,1.0,1.0,0.0,0.0,0.0
50%,1.0,1.0,0.0,1.0,0.0
75%,1.0,1.0,0.0,1.0,1.0
max,1.0,1.0,1.0,2.0,2.0


In [8]:
res =\
(
    assign_model(kmeans)
    .reset_index()
    .assign(
        user_id = lambda x: x.index_.str.split(" ", expand=False).str[0],
        date_ = lambda x: x.index_.str.split(" ", expand=False).str[1]
    )
    .drop(columns=['index_'])
    .merge(imp_df, how='right', on=['user_id', 'date_'])
    # .assign(
    #     impressions_count = lambda x: x.impressions_count.fillna(0),
    #     read_count = lambda x: x.read_count.fillna(0),
    # )
)

In [9]:
res.describe()

,M1_core_count,M1_ppob_count,M1_acc_count,total_customer,is_referrer,age_on_platform_days,send_count,impressions_count,read_count,activated_count,converted_count
count,100513.0,100513.0,126563.0,67245.0,127432.0,127432.0,127650.0,127650.0,127650.0,127650.0,127650.0
mean,0.902582,1.461422,1.513373,38.35732,0.051463,5.56273,1.0,0.918841,0.106761,0.677282,0.317705
std,1.574336,1.77404,2.116453,86.859556,0.220941,1.299359,0.0,0.273081,0.30881,0.467736,0.465687
min,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,3.0,0.0,4.70048,1.0,1.0,0.0,0.0,0.0
50%,0.0,0.693147,0.0,11.0,0.0,6.100319,1.0,1.0,0.0,1.0,0.0
75%,1.098612,2.890372,3.258097,38.0,0.0,6.568078,1.0,1.0,0.0,1.0,1.0
max,8.2247,8.548886,9.481817,2907.0,1.0,7.257708,1.0,1.0,1.0,2.0,2.0


In [10]:
res.groupby(['Cluster']).agg('mean')

,M1_core_count,M1_ppob_count,M1_acc_count,total_customer,is_referrer,age_on_platform_days,send_count,impressions_count,read_count,activated_count,converted_count
Cluster,,,,,,,,,,,
Cluster 0,1.035717,1.796392,0.503453,37.453552,0.101519,5.091922,1.0,0.908074,0.144684,0.707834,0.453637
Cluster 1,0.037033,1.269133,2.647314,9.092278,0.021941,5.486933,1.0,0.943267,0.071339,0.682766,0.213976
Cluster 2,1.921712,2.762193,0.715554,43.298626,0.093324,5.733997,1.0,0.945519,0.179418,0.895494,0.687994
Cluster 3,0.079418,0.923923,2.286065,8.320303,0.041912,6.414967,1.0,0.928375,0.066808,0.645002,0.203741
Cluster 4,0.665,0.836295,0.431849,10.957323,0.039723,5.19903,1.0,0.876666,0.102059,0.559181,0.276221
Cluster 5,3.713483,3.376627,0.449462,157.606485,0.188989,5.91008,1.0,0.92992,0.260453,0.912721,0.683108


In [11]:
res.groupby(['date_', 'Cluster']).agg('mean')

M1_core_count  M1_ppob_count  M1_acc_count  \
date_      Cluster                                                 
2023-05-01 Cluster 0       1.247444       1.538726      0.493922   
           Cluster 1        0.04848       1.114907      2.117494   
           Cluster 2        2.28597       2.573692       0.60476   
           Cluster 3       0.074886       0.796598      2.000952   
           Cluster 4        0.91926       0.966445      0.427107   
           Cluster 5       3.997727       2.950307      0.386477   
2023-06-01 Cluster 0        1.04197       1.973334      0.562746   
           Cluster 1       0.036316       1.405014      2.888659   
           Cluster 2       1.893578       2.811379      0.723755   
           Cluster 3       0.082954       1.082339      2.501551   
           Cluster 4       0.798912       0.980445      0.460457   
           Cluster 5       3.756575       3.381417      0.440263   
2023-07-01 Cluster 0       1.006001       1.816533      0.491215   
           Cluster 1       0.037691       1.293592      2.838159   
           Cluster 2       1.845688       2.848383      0.758385   
           Cluster 3       0.083117       0.939305       2.39769   
           Cluster 4        0.58316       0.776387      0.435384   
           Cluster 5       3.631264       3.508487      0.482708   
2023-08-01 Cluster 0       0.904569       1.845184      0.481509   
           Cluster 1       0.027424       1.270416      2.751338   
           Cluster 2       1.748159       2.782577      0.750602   
           Cluster 3       0.078655       0.936448      2.374348   
           Cluster 4       0.459976       0.696575      0.412578   
           Cluster 5       3.583614       3.513551      0.463666   

                      total_customer  is_referrer  age_on_platform_days  \
date_      Cluster                                                        
2023-05-01 Cluster 0       35.684834     0.109929              5.213281   
           Cluster 1        9.178789     0.022534              5.685275   
           Cluster 2       49.460984     0.103353              5.736063   
           Cluster 3        7.635851     0.039938              6.338141   
           Cluster 4       11.937248     0.042979              5.144285   
           Cluster 5      166.991539     0.209703              5.938702   
2023-06-01 Cluster 0       36.307692     0.093933              4.849729   
           Cluster 1        8.861149     0.020379              5.217383   
           Cluster 2       43.260937       0.0884              5.667047   
           Cluster 3        8.706331      0.04254              6.395307   
           Cluster 4       10.673993     0.033271              4.918182   
           Cluster 5       158.85124     0.188636              5.843031   
2023-07-01 Cluster 0       38.956432     0.106129              5.162511   
           Cluster 1        8.877778     0.022096              5.458313   
           Cluster 2       39.480948      0.09031              5.718517   
           Cluster 3        8.447925      0.04212              6.449897   
           Cluster 4       10.021451     0.040946              5.232388   
           Cluster 5      149.801445      0.17862              5.892165   
2023-08-01 Cluster 0       38.328358     0.096306              5.102595   
           Cluster 1        9.430637     0.022519              5.551056   
           Cluster 2        42.27165     0.092577              5.798089   
           Cluster 3        8.734244     0.043792              6.500106   
           Cluster 4       11.109768     0.040526              5.401553   
           Cluster 5      158.343387     0.186198               5.95731   

                      send_count  impressions_count  read_count  \
date_      Cluster                                                
2023-05-01 Cluster 0         1.0           0.911348    0.113475   
           Cluster 1         1.0           0.905017    0.066582   
           Cluster 2         1.0  

In [12]:
res.groupby([ 'date_', 'Cluster']).agg('count')

kyc_tier  loyalty_tier  M1_core_count  M1_ppob_count  \
date_      Cluster                                                           
2023-05-01 Cluster 0       564           564            514            514   
           Cluster 1     11760         11760           6023           6023   
           Cluster 2      2535          2535           2535           2535   
           Cluster 3      5133          5133           3657           3657   
           Cluster 4      8888          8888           8689           8689   
           Cluster 5      1917          1917           1917           1917   
2023-06-01 Cluster 0       511           511            481            481   
           Cluster 1     10501         10501           5601           5601   
           Cluster 2      2681          2681           2681           2681   
           Cluster 3      3291          3291           2443           2443   
           Cluster 4      8025          8025           7934           7934   
           Cluster 5      2200          2200           2200           2200   
2023-07-01 Cluster 0       669           669            630            630   
           Cluster 1     11948         11948           6484           6484   
           Cluster 2      3189          3189           3189           3189   
           Cluster 3      3680          3680           2744           2744   
           Cluster 4     10990         10990          10862          10862   
           Cluster 5      2928          2928           2928           2928   
2023-08-01 Cluster 0       758           758            710            710   
           Cluster 1     13100         13100           7201           7201   
           Cluster 2      3489          3489           3489           3489   
           Cluster 3      3882          3882           2907           2907   
           Cluster 4     11721         11721          11622          11622   
           Cluster 5      3072          3072           3072           3072   

                      M1_acc_count  total_customer  is_referrer  \
date_      Cluster                                                
2023-05-01 Cluster 0           542             422          564   
           Cluster 1         11539            3451        11760   
           Cluster 2          2535            2499         2535   
           Cluster 3          5126            1774         5133   
           Cluster 4          8786            6454         8888   
           Cluster 5          1917            1891         1917   
2023-06-01 Cluster 0           497             390          511   
           Cluster 1         10381            3673        10501   
           Cluster 2          2681            2583         2681   
           Cluster 3          3289            1311         3291   
           Cluster 4          7983            5460         8025   
           Cluster 5          2200            2178         2200   
2023-07-01 Cluster 0           659             482          669   
           Cluster 1         11826            3870        11948   
           Cluster 2          3189            3123         3189   
           Cluster 3          3675            1277         3680   
           Cluster 4         10942            5967        10990   
           Cluster 5          2928            2906         2928   
2023-08-01 Cluster 0           740             536          758   
           Cluster 1         12990            4080        13100   
           Cluster 2          3489            3291         3489   
           Cluster 3          3880            1317         3882   
           Cluster 4         11697            5293        11721   
           Cluster 5          3072            3017         3072   

                      acquisition_channel  user_segment_sub_bucket  \
date_      Cluster                                                   
2023-05-01 Cluster 0                  564                      564   
           Cluster 1                1176

In [13]:
# res.to_csv('result/result_v3.csv')